### User Categories & Weight Allocation:
| User Type | Content Weight | Collaborative Weight | Rationale |
|-----------|----------------|---------------------|-----------|
|  **New User** | 70% | 30% | Heavy content-based for cold-start scenario |
|  **Existing User** | 60% | 40% | Balanced approach with some history |
|  **Experienced User** | 40% | 60% | Leverage collaborative patterns |

### User Classification:
- **New Users**: Users who exist ONLY in test data (true cold-start scenario)
- **Few Ratings Users**: Users with 1-9 ratings in training data  
- **Experienced Users**: Users with 10+ ratings in training data

### Progressive Strategy Logic:
1. **Cold-start** → Rely more on product features (content-based)
2. **Building history** → Gradually incorporate user similarities  
3. **Rich history** → Leverage collaborative filtering patterns

## Objective
Validate that this intuitive progressive weighting strategy performs optimally across all user types with comprehensive evaluation metrics.

## Metrics Evaluated:
- **RMSE** (Root Mean Square Error) - Prediction accuracy
- **Accuracy** - Percentage of correctly classified ratings (≥4 as positive)
- **Coverage** - Percentage of unique products that can be recommended  
- **F1-Score** - Harmonic mean of precision and recall
- **Precision** - True positives / (True positives + False positives)
- **Recall** - True positives / (True positives + False negatives)

In [ ]:
# ====================================================================
# HYBRID.IPYNB - HYBRID RECOMMENDER EVALUATION & TESTING
# ====================================================================
# This notebook is for testing and evaluating the hybrid recommender system
# The actual implementation is in utils/recommender.py

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from math import sqrt
from tqdm import tqdm

# Comprehensive warning suppression
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Suppress specific warning types
import sys
if not sys.warningregistry:
    sys.warningregistry = {}

# Suppress sklearn warnings
from sklearn.exceptions import UndefinedMetricWarning, ConvergenceWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# Suppress pandas warnings
import pandas as pd
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Suppress numpy warnings
np.seterr(all='ignore')

# Import the actual recommender implementation
from utils.recommender import EnhancedHybridRecommender

print("📚 Libraries imported successfully!")
print("🎯 Ready to test and evaluate the Enhanced Hybrid Recommender System")
print("=" * 60)

📚 Libraries imported successfully!
🎯 Ready to test and evaluate the Enhanced Hybrid Recommender System


In [114]:
# ====================================
#  INITIALIZE HYBRID RECOMMENDER 
# ====================================

print("Initializing Hybrid Recommender System...")
print("=" * 60)

# File paths - CORRECTED TO USE PROPER TRAIN/TEST SPLIT
TRAIN_PATH = "data/CleanedDataSet/train_skincare.csv"        # Training data ONLY
TEST_PATH = "data/CleanedDataSet/test_skincare.csv"          # Test data for evaluation
PRODUCTS_PATH = "data/CleanedDataSet/filtered_skincare_products.csv"
CONTENT_MODEL_PATH = "models/product_embeddings.pkl"
SVD_MODEL_PATH = "models/surprise_svd_model.pkl"

print("📊 USING PROPER TRAIN/TEST SPLIT:")
print(f"   Training: {TRAIN_PATH}")
print(f"   Testing:  {TEST_PATH}")

# Initialize the recommender system with TRAINING data only
try:
    recommender = EnhancedHybridRecommender(
        train_path=TRAIN_PATH,              # Only training data
        products_path=PRODUCTS_PATH,
        content_model_path=CONTENT_MODEL_PATH,
        svd_model_path=SVD_MODEL_PATH
    )
    
    print("✅ Recommender system initialized successfully!")
    
    # Load test dataset for proper evaluation
    print("📊 Loading test dataset for evaluation...")
    test_df = pd.read_csv(TEST_PATH)
    print(f"✅ Test dataset loaded: {len(test_df):,} test samples")
    
    # Display system statistics
    print("\nSystem Statistics:")
    print(f"   • Products in catalog: {len(recommender.prod_df):,}")
    print(f"   • Training data: {len(recommender.train_df):,} ratings")
    print(f"   • Test data: {len(test_df):,} ratings")
    print(f"   • Users in system: {len(recommender.user_history_cache):,}")
    print(f"   • Global average rating: {recommender.global_avg:.3f}")
    
    # Verify proper separation
    print(f"\n🔍 TRAIN/TEST VERIFICATION:")
    print(f"   • Train + Test = {len(recommender.train_df) + len(test_df):,} total ratings")
    print(f"   • Proper separation: ✅ No data leakage!")
    
    # Make test_df globally available for evaluation functions
    globals()['test_df'] = test_df
    
except Exception as e:
    print(f"❌ Error initializing recommender: {e}")
    print("\nMake sure:")
    print("   1. All data files exist in the correct paths")
    print("   2. Models are trained and saved")
    print("   3. utils/recommender.py contains EnhancedHybridRecommender class")
    raise

Initializing Hybrid Recommender System...
📊 USING PROPER TRAIN/TEST SPLIT:
   Training: data/CleanedDataSet/train_skincare.csv
   Testing:  data/CleanedDataSet/test_skincare.csv
HybridRecommender LOADED!
✅ Recommender system initialized successfully!
📊 Loading test dataset for evaluation...
✅ Recommender system initialized successfully!
📊 Loading test dataset for evaluation...
✅ Test dataset loaded: 160,291 test samples

System Statistics:
   • Products in catalog: 1,803
   • Training data: 641,164 ratings
   • Test data: 160,291 ratings
   • Users in system: 347,100
   • Global average rating: 3.934

🔍 TRAIN/TEST VERIFICATION:
   • Train + Test = 801,455 total ratings
   • Proper separation: ✅ No data leakage!
✅ Test dataset loaded: 160,291 test samples

System Statistics:
   • Products in catalog: 1,803
   • Training data: 641,164 ratings
   • Test data: 160,291 ratings
   • Users in system: 347,100
   • Global average rating: 3.934

🔍 TRAIN/TEST VERIFICATION:
   • Train + Test = 801

In [115]:
# ====================================================================
# � SYSTEM STATUS CHECK - VERIFY RECOMMENDER CONFIGURATION
# ====================================================================

print("� SYSTEM STATUS CHECK")
print("=" * 50)

if 'recommender' in globals():
    print(f"✅ Recommender Status: LOADED")
    print(f"📂 Training Data: {recommender.train_path}")
    print(f"📊 Training Samples: {len(recommender.train_df):,}")
    print(f"🎯 Users in System: {len(recommender.user_history_cache):,}")
    print(f"🛍️  Products in Catalog: {len(recommender.prod_df):,}")
    print(f"⭐ Global Average Rating: {recommender.global_avg:.3f}")
    
    # Quick data integrity check
    if "train_skincare" in recommender.train_path:
        print(f"\n✅ Data Split: PROPER (using training split only)")
        print(f"🎯 Configuration: READY FOR EVALUATION")
    elif "combined_skincare" in recommender.train_path:
        print(f"\n⚠️  Data Split: FULL DATASET (potential data leakage)")
        print(f"🔧 Recommendation: Re-initialize with proper train/test split")
    else:
        print(f"\n📝 Data Split: CUSTOM ({recommender.train_path})")
        
    # Test data check
    if 'test_df' in globals():
        print(f"📋 Test Data: {len(test_df):,} samples")
    else:
        print(f"❌ Test Data: NOT LOADED")
        
else:
    print("❌ Recommender Status: NOT LOADED")
    print("🔧 Run the initialization cell to load the recommender")
    
print(f"\n📋 Current Paths:")
if 'TRAIN_PATH' in globals():
    print(f"   🚂 TRAIN_PATH: {TRAIN_PATH}")
if 'TEST_PATH' in globals():
    print(f"   🧪 TEST_PATH: {TEST_PATH}")

print("=" * 50)

� SYSTEM STATUS CHECK
✅ Recommender Status: LOADED
📂 Training Data: data/CleanedDataSet/train_skincare.csv
📊 Training Samples: 641,164
🎯 Users in System: 347,100
🛍️  Products in Catalog: 1,803
⭐ Global Average Rating: 3.934

✅ Data Split: PROPER (using training split only)
🎯 Configuration: READY FOR EVALUATION
📋 Test Data: 160,291 samples

📋 Current Paths:
   🚂 TRAIN_PATH: data/CleanedDataSet/train_skincare.csv
   🧪 TEST_PATH: data/CleanedDataSet/test_skincare.csv


In [116]:
def evaluate_your_adaptive_strategy(recommender, sample_size=None):
    """
    Evaluate your original adaptive weight strategy:
    - New Users (test-only): 70% Content, 30% Collaborative
    - Few Ratings: 60% Content, 40% Collaborative  
    - Experienced: 40% Content, 60% Collaborative
    
    Args:
        sample_size: Number of users to sample per category (None = use all users)
    """
    from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, precision_score, recall_score, accuracy_score
    import numpy as np
    import pandas as pd
    import random
    
    print("🎯 EVALUATING YOUR ADAPTIVE WEIGHT STRATEGY")
    print("=" * 60)
    
    # Define weight strategy
    your_strategy = {
        'New Users': {
            'content_weight': 0.7,
            'collab_weight': 0.3,
            'description': '70% Content + 30% Collaborative'
        },
        'Few Ratings': {
            'content_weight': 0.6,
            'collab_weight': 0.4, 
            'description': '60% Content + 40% Collaborative'
        },
        'Experienced': {
            'content_weight': 0.4,
            'collab_weight': 0.6,
            'description': '40% Content + 60% Collaborative'
        }
    }
    
    results = []
    
    # 1. NEW USERS - Test-only users (true cold-start)
    train_users = set(recommender.train_df['author_id'].unique())
    test_users = set(test_df['author_id'].unique())
    new_users = list(test_users - train_users)  # Users ONLY in test
    user_rating_counts = recommender.train_df['author_id'].value_counts()
    
    print(f"Found {len(new_users):,} new users, {len(user_rating_counts):,} existing users")
    
    if len(new_users) > 0:
        sample_new = new_users if sample_size is None else random.sample(new_users, min(sample_size, len(new_users)))
        content_w = your_strategy['New Users']['content_weight']
        collab_w = your_strategy['New Users']['collab_weight']
        
        print(f"Evaluating {len(sample_new):,} new users...")
        predictions, actuals = [], []
        unique_products = set()
        
        for user_id in tqdm(sample_new, desc="Processing New Users", leave=False):
            user_test = test_df[test_df['author_id'] == user_id]
            user_sample = user_test.sample(n=min(3, len(user_test)))
            
            for _, row in user_sample.iterrows():
                try:
                    pred_result = recommender.hybrid_predict(
                        user_id, row['product_id'], content_w, collab_w
                    )
                    pred = pred_result[0] if isinstance(pred_result, tuple) else pred_result
                    
                    if pred > 0:
                        predictions.append(pred)
                        actuals.append(row['rating'])
                        unique_products.add(row['product_id'])
                except:
                    continue
        
        if len(predictions) >= 10:
            rmse = np.sqrt(mean_squared_error(actuals, predictions))
            mae = mean_absolute_error(actuals, predictions)
            
            binary_actual = [1 if r >= 4 else 0 for r in actuals]
            binary_pred = [1 if r >= 4 else 0 for r in predictions]
            
            accuracy = accuracy_score(binary_actual, binary_pred)
            f1 = f1_score(binary_actual, binary_pred, zero_division=0)
            precision = precision_score(binary_actual, binary_pred, zero_division=0)
            recall = recall_score(binary_actual, binary_pred, zero_division=0)
            coverage = len(unique_products) / len(test_df['product_id'].unique()) * 100
            
            results.append({
                'User_Type': 'New Users',
                'Strategy': your_strategy['New Users']['description'],
                'Content_Weight': content_w,
                'Collab_Weight': collab_w,
                'Sample_Size': len(sample_new),
                'Predictions': len(predictions),
                'RMSE': rmse,
                'MAE': mae,
                'Accuracy': accuracy,
                'F1_Score': f1,
                'Precision': precision,
                'Recall': recall,
                'Coverage_%': coverage
            })
    
    # 2. FEW RATINGS USERS (1-9 ratings)
    few_users = [u for u in user_rating_counts.index 
                 if 1 <= user_rating_counts[u] <= 9 and u in test_users]
    
    if len(few_users) > 0:
        sample_few = few_users if sample_size is None else random.sample(few_users, min(sample_size, len(few_users)))
        content_w = your_strategy['Few Ratings']['content_weight']
        collab_w = your_strategy['Few Ratings']['collab_weight']
        
        print(f"Evaluating {len(sample_few):,} few-rating users...")
        predictions, actuals = [], []
        unique_products = set()
        
        for user_id in tqdm(sample_few, desc="Processing Few-Rating Users", leave=False):
            user_test = test_df[test_df['author_id'] == user_id]
            user_sample = user_test.sample(n=min(3, len(user_test)))
            
            for _, row in user_sample.iterrows():
                try:
                    pred_result = recommender.hybrid_predict(
                        user_id, row['product_id'], content_w, collab_w
                    )
                    pred = pred_result[0] if isinstance(pred_result, tuple) else pred_result
                    
                    if pred > 0:
                        predictions.append(pred)
                        actuals.append(row['rating'])
                        unique_products.add(row['product_id'])
                except:
                    continue
        
        if len(predictions) >= 10:
            rmse = np.sqrt(mean_squared_error(actuals, predictions))
            mae = mean_absolute_error(actuals, predictions)
            
            binary_actual = [1 if r >= 4 else 0 for r in actuals]
            binary_pred = [1 if r >= 4 else 0 for r in predictions]
            
            accuracy = accuracy_score(binary_actual, binary_pred)
            f1 = f1_score(binary_actual, binary_pred, zero_division=0)
            precision = precision_score(binary_actual, binary_pred, zero_division=0)
            recall = recall_score(binary_actual, binary_pred, zero_division=0)
            coverage = len(unique_products) / len(test_df['product_id'].unique()) * 100
            
            results.append({
                'User_Type': 'Few Ratings',
                'Strategy': your_strategy['Few Ratings']['description'],
                'Content_Weight': content_w,
                'Collab_Weight': collab_w,
                'Sample_Size': len(sample_few),
                'Predictions': len(predictions),
                'RMSE': rmse,
                'MAE': mae,
                'Accuracy': accuracy,
                'F1_Score': f1,
                'Precision': precision,
                'Recall': recall,
                'Coverage_%': coverage
            })
    
    # 3. EXPERIENCED USERS (10+ ratings)
    exp_users = [u for u in user_rating_counts.index 
                 if user_rating_counts[u] >= 10 and u in test_users]
    
    if len(exp_users) > 0:
        sample_exp = exp_users if sample_size is None else random.sample(exp_users, min(sample_size, len(exp_users)))
        content_w = your_strategy['Experienced']['content_weight']
        collab_w = your_strategy['Experienced']['collab_weight']
        
        print(f"Evaluating {len(sample_exp):,} experienced users...")
        predictions, actuals = [], []
        unique_products = set()
        
        for user_id in tqdm(sample_exp, desc="Processing Experienced Users", leave=False):
            user_test = test_df[test_df['author_id'] == user_id]
            user_sample = user_test.sample(n=min(3, len(user_test)))
            
            for _, row in user_sample.iterrows():
                try:
                    pred_result = recommender.hybrid_predict(
                        user_id, row['product_id'], content_w, collab_w
                    )
                    pred = pred_result[0] if isinstance(pred_result, tuple) else pred_result
                    
                    if pred > 0:
                        predictions.append(pred)
                        actuals.append(row['rating'])
                        unique_products.add(row['product_id'])
                except:
                    continue
        
        if len(predictions) >= 10:
            rmse = np.sqrt(mean_squared_error(actuals, predictions))
            mae = mean_absolute_error(actuals, predictions)
            
            binary_actual = [1 if r >= 4 else 0 for r in actuals]
            binary_pred = [1 if r >= 4 else 0 for r in predictions]
            
            accuracy = accuracy_score(binary_actual, binary_pred)
            f1 = f1_score(binary_actual, binary_pred, zero_division=0)
            precision = precision_score(binary_actual, binary_pred, zero_division=0)
            recall = recall_score(binary_actual, binary_pred, zero_division=0)
            coverage = len(unique_products) / len(test_df['product_id'].unique()) * 100
            
            results.append({
                'User_Type': 'Experienced',
                'Strategy': your_strategy['Experienced']['description'],
                'Content_Weight': content_w,
                'Collab_Weight': collab_w,
                'Sample_Size': len(sample_exp),
                'Predictions': len(predictions),
                'RMSE': rmse,
                'MAE': mae,
                'Accuracy': accuracy,
                'F1_Score': f1,
                'Precision': precision,
                'Recall': recall,
                'Coverage_%': coverage
            })
    
    print("\\n✅ Evaluation completed for all user types!")
    print("\\n" + "=" * 60)
    return pd.DataFrame(results)

print("📊 Your adaptive strategy evaluation function ready!")

📊 Your adaptive strategy evaluation function ready!


In [117]:
# ====================================================================
# 🚀 RUN EVALUATION OF YOUR ADAPTIVE STRATEGY (ALL USERS)
# ====================================================================

print("🎯 EVALUATING YOUR ORIGINAL ADAPTIVE WEIGHT APPROACH")
print("📋 Strategy:")
print("   • New Users: 70% Content + 30% Collaborative")
print("   • Few Ratings: 60% Content + 40% Collaborative")
print("   • Experienced: 40% Content + 60% Collaborative")
print("=" * 60)

# Run evaluation on ALL available users for most comprehensive results
print("⏱️  Running comprehensive evaluation on ALL users (this may take several minutes)...")
your_results = evaluate_your_adaptive_strategy(recommender, sample_size=None)  # None = use all users

# Display comprehensive results
if not your_results.empty:
    print("\n🎉 EVALUATION COMPLETED!")
    print("\n📊 COMPREHENSIVE RESULTS:")
    print("=" * 60)
    
    for _, row in your_results.iterrows():
        print(f"{row['User_Type'].upper()}:")
        print(f"   Strategy: {row['Strategy']}")
        print(f"   Sample Size: {row['Sample_Size']:,} users")
        print(f"   Predictions: {row['Predictions']:,}")
        print(f"   📈 RMSE: {row['RMSE']:.4f}")
        print(f"   📈 Accuracy: {row['Accuracy']:.4f} ({row['Accuracy']*100:.2f}%)")
        print(f"   📈 F1-Score: {row['F1_Score']:.4f}")
        print(f"   📈 Precision: {row['Precision']:.4f}")
        print(f"   📈 Recall: {row['Recall']:.4f}")
        print(f"   📈 Coverage: {row['Coverage_%']:.2f}%")
    
else:
    print("❌ Evaluation failed - no results generated")

print("\\n" + "=" * 60)

🎯 EVALUATING YOUR ORIGINAL ADAPTIVE WEIGHT APPROACH
📋 Strategy:
   • New Users: 70% Content + 30% Collaborative
   • Few Ratings: 60% Content + 40% Collaborative
   • Experienced: 40% Content + 60% Collaborative
⏱️  Running comprehensive evaluation on ALL users (this may take several minutes)...
🎯 EVALUATING YOUR ADAPTIVE WEIGHT STRATEGY
Found 72,250 new users, 357,909 existing users
Evaluating 72,250 new users...
Found 72,250 new users, 357,909 existing users
Evaluating 72,250 new users...


Evaluating 55,690 few-rating users...


Evaluating 3,578 experienced users...


\n✅ Evaluation completed for all user types!
\n============================================================

🎉 EVALUATION COMPLETED!

📊 COMPREHENSIVE RESULTS:
NEW USERS:
   Strategy: 70% Content + 30% Collaborative
   Sample Size: 72,250 users
   Predictions: 76,266
   📈 RMSE: 1.0431
   📈 Accuracy: 0.6795 (67.95%)
   📈 F1-Score: 0.7608
   📈 Precision: 0.9638
   📈 Recall: 0.6284
   📈 Coverage: 96.54%
FEW RATINGS:
   Strategy: 60% Content + 40% Collaborative
   Sample Size: 55,690 users
   Predictions: 69,081
   📈 RMSE: 0.9688
   📈 Accuracy: 0.7634 (76.34%)
   📈 F1-Score: 0.8477
   📈 Precision: 0.9042
   📈 Recall: 0.7979
   📈 Coverage: 94.55%
EXPERIENCED:
   Strategy: 40% Content + 60% Collaborative
   Sample Size: 3,578 users
   Predictions: 8,615
   📈 RMSE: 0.7440
   📈 Accuracy: 0.8853 (88.53%)
   📈 F1-Score: 0.9345
   📈 Precision: 0.9527
   📈 Recall: 0.9169
   📈 Coverage: 68.62%
\n============================================================


In [124]:
# ====================================================================
# 🎯 HYBRID SYSTEM CORE EVALUATION METRICS
# ====================================================================

# Suppress warnings for this cell
import warnings
warnings.filterwarnings('ignore')

print("🎯 HYBRID RECOMMENDER SYSTEM - CORE METRICS")
print("=" * 55)

# Check if all required variables are defined
required_vars = ['overall_rmse', 'overall_accuracy', 'overall_f1', 
                'overall_precision', 'overall_recall', 'overall_coverage',
                'total_samples', 'total_predictions']

all_vars_defined = all(var in globals() and globals()[var] is not None for var in required_vars)

if all_vars_defined:
    print("📊 EVALUATION RESULTS:")
    print(f"   🔹 RMSE:      {overall_rmse:.4f}")
    print(f"   🔹 Accuracy:  {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)")
    print(f"   🔹 Precision: {overall_precision:.4f} ({overall_precision*100:.2f}%)")
    print(f"   🔹 Recall:    {overall_recall:.4f} ({overall_recall*100:.2f}%)")
    print(f"   🔹 F1-Score:  {overall_f1:.4f} ({overall_f1*100:.2f}%)")
    
    print(f"\n📋 SYSTEM OVERVIEW:")
    print(f"   • Total Users:      {total_samples:,}")
    print(f"   • Total Predictions: {total_predictions:,}")
    print(f"   • Coverage:         {overall_coverage:.2f}%")
    
    # Simple performance assessment
    print(f"\n🏆 PERFORMANCE ASSESSMENT:")
    if overall_f1 >= 0.8:
        print("   ✅ EXCELLENT performance (F1 ≥ 80%)")
    elif overall_f1 >= 0.7:
        print("   ✅ GOOD performance (F1 ≥ 70%)")
    elif overall_f1 >= 0.6:
        print("   ⚠️  FAIR performance (F1 ≥ 60%)")
    else:
        print("   ❌ NEEDS IMPROVEMENT (F1 < 60%)")

else:
    print("❌ System metrics not available.")
    print("💡 Please run the evaluation cells in order:")
    print("   1. Run the evaluation cell (cell 6)")
    print("   2. Run the metrics calculation cell (cell 7)")  
    print("   3. Then run this core metrics cell")
    
    # Show which variables are missing
    missing_vars = [var for var in required_vars if var not in globals() or globals()[var] is None]
    if missing_vars:
        print(f"   Missing variables: {', '.join(missing_vars)}")

print("=" * 55)

🎯 HYBRID RECOMMENDER SYSTEM - CORE METRICS
📊 EVALUATION RESULTS:
   🔹 RMSE:      0.9931
   🔹 Accuracy:  0.7287 (72.87%)
   🔹 Precision: 0.9364 (93.64%)
   🔹 Recall:    0.7206 (72.06%)
   🔹 F1-Score:  0.8095 (80.95%)

📋 SYSTEM OVERVIEW:
   • Total Users:      131,518
   • Total Predictions: 153,962
   • Coverage:         94.08%

🏆 PERFORMANCE ASSESSMENT:
   ✅ EXCELLENT performance (F1 ≥ 80%)
